In [8]:
!pip install git+https://github.com/alibaba/TinyNeuralNetwork.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-req-build-lhes23rn
  Running command git clone -q https://github.com/alibaba/TinyNeuralNetwork.git /tmp/pip-req-build-lhes23rn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for TinyNeuralNetwork: filename=TinyNeuralNetwork-0.1.0.20220929231908+a521163753e4b201a4a31d3cb82e5aa1fe328d93-py3-none-any.whl size=352164 sha256=4dc00262561963e130e096be1d3dccf54d4c7164f601140b0974d12545c0bc9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkf9udsc/wheels/64/6b/f5/9cf69de054ba0de53e572bc2f13988a664f3dc9623e6f0825d
Successfully built TinyNeuralNetwork
  Attempting uninstall: TinyNeuralNetwork
    Found existing installation: TinyNeuralNetwork 0.1.0.20220929231341+a521163753e4b201a4a31d3cb82e5aa1fe328d93
    Uninstalling TinyNeuralNet

In [9]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fgx6fwr7
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-fgx6fwr7


In [10]:
!git clone https://github.com/usefulsensors/openai-whisper.git

Cloning into 'openai-whisper'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 33 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


## Convert from pytorch to onnx

In [11]:
import whisper
import torch

tiny_model = whisper.load_model("tiny")
torch.onnx.export(tiny_model.encoder, torch.randn(1,80,3000).to("cuda"), "/content/whisper-encoder.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258]]).to("cuda"), torch.randn(1,384,384).to("cuda")), "/content/whisper-decoder_main.onnx")
torch.onnx.export(tiny_model.decoder, (torch.tensor([[50258, 50259, 50359]]).to("cuda"), torch.randn(1, 384, 384).to("cuda")), "/content/whisper-decoder_language.onnx")

## Convert from pytorch to tflite using Tinynn

In [12]:
import whisper
import torch

import torch.nn as nn
from tinynn.converter import TFLiteConverter
from tinynn.graph.quantization.quantizer import PostQuantizer
from tinynn.graph.tracer import model_tracer
from tinynn.util.train_util import DLContext, get_device, train

model = whisper.load_model("tiny")
model.eval()
audio = whisper.load_audio("/content/openai-whisper/filename.mp3")
audio = whisper.pad_or_trim(audio)
print(audio.shape)
# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)
print(mel.shape)
quantizer = PostQuantizer(model, mel, work_dir='out', config={'asymmetric': True, 'per_tensor': False})
#qat_model = quantizer.quantize()


(480000,)
torch.Size([80, 3000])
